<a href="https://colab.research.google.com/github/komo135/forex-prediction/blob/master/ipynb/rl/qr_dqn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Google ドライブをマウントするには、このセルを実行してください。
from google.colab import drive
drive.mount('/content/drive')

%cd drive/My Drive

Mounted at /content/drive
/content/drive/My Drive


In [0]:
%tensorflow_version 2.x
import tensorflow as tf
import numpy as np

In [15]:
%cd /content/drive/My Drive

try:
  import imp
  imp.reload(qr_dqn)
except:
  import qr_dqn

agent = qr_dqn.Agent(restore=True, n=3, lr=1e-3)

/content/drive/My Drive


In [0]:
%cd /content
agent.run(train=True)

/content
action probability: buy=0.2916666666666667, sell=0.3020833333333333, hold=0.40625
epoch: 87000, total assets:2134.7218506003146, growth_rate:0.021347218506003147

action probability: buy=0.25, sell=0.2916666666666667, hold=0.4583333333333333
epoch: 87005, total assets:8765.080222878933, growth_rate:0.08765080222878932

action probability: buy=0.21875, sell=0.3958333333333333, hold=0.3854166666666667
epoch: 87010, total assets:10551.548784993714, growth_rate:0.10551548784993714

action probability: buy=0.375, sell=0.20833333333333334, hold=0.4166666666666667
epoch: 87015, total assets:5823.101660042932, growth_rate:0.05823101660042932

action probability: buy=0.25, sell=0.34375, hold=0.40625
epoch: 87020, total assets:40071.24125462755, growth_rate:0.40071241254627554

action probability: buy=0.2708333333333333, sell=0.3333333333333333, hold=0.3958333333333333
epoch: 87025, total assets:90831.53191368667, growth_rate:0.9083153191368667

action probability: buy=0.364583333333333

In [0]:
import numpy as np
tree_idx, replay = agent.memory.sample(128)

states = np.array([a[0][0] for a in replay], np.float32)
new_states = np.array([a[0][3] for a in replay], np.float32)
actions = np.array([a[0][1] for a in replay]).reshape((-1, 1))
rewards = np.array([a[0][2] for a in replay], np.float32).reshape((-1, 1))

q = agent.model(states)
target_q = agent.target_model(new_states).numpy()
arg_q = np.sum(agent.model(new_states).numpy(), -1).reshape((-1,2))
arg_q = np.argmax(arg_q, -1)
# arg_q = [np.argmax(i) if 0.05 < np.random.rand() else np.random.randint(2) for i in arg_q]

q_backup = q.numpy()

for i in range(q.shape[0]):
  q_backup[i, actions[i]] = rewards[i] + agent.gamma * target_q[i, arg_q[i]]


error = tf.abs(q_backup - q)
# q_error = tf.maximum(self.tau * error, (self.tau - 1) * error)
loss = tf.where(error < 2, error ** 2 * .5, 2 * error - 0.5 * 2 ** 2)
loss *= tf.abs(agent.tau - tf.stop_gradient(tf.cast((q_backup - q) < 0, tf.float32)))
# loss = agent.huber_loss(q_backup, q)
# loss = tf.abs(agent.tau - tf.stop_gradient(tf.cast((q_backup - q) < 0, tf.float32))) * loss
# loss = tf.reduce_sum(loss, 2)

# print(q)

In [14]:
np.sum(agent.model.predict_on_batch(new_states), -1).astype(np.int32)

array([[-138978,  -42136, -106470],
       [ -65316,  -56299, -171827],
       [ -39747,  -30925, -131847],
       [-117627,  -40959, -104135],
       [ -49723,  -72893,  -23653],
       [-101974,  -19158,  -92798],
       [ -88921,  -44127,  -64252],
       [  -3621,  -89915,  -26602],
       [ -34899,  -27829,  -25161],
       [ -41012, -251874,  -84328],
       [ -17213,  -71030,  -52134],
       [-106178,  -76908,  -54928],
       [ -60091,  -56785,  -82189],
       [-138154,  -92672,  -63639],
       [ -16683, -185027,  -50387],
       [ -41461,  -75815,  -15053],
       [ -93275,   50476,  -97672],
       [ -83211,  -32529,  -78591],
       [-111472,  -52040,  -71190],
       [ -72873, -100496,   -8255],
       [-116064,   30547,  -69601],
       [-107772,  -56543,  -79324],
       [ -62770,  -62183,  -43893],
       [-243135,  -21294, -124733],
       [ -92595,  -65489,  -43723],
       [-112587, -141119,     953],
       [ -75065, -111467,  -72057],
       [ -64808,  -33368,  -

In [0]:
np.sum(q_backup, -1).astype(np.int32)

In [0]:
agent.run(train=False)

In [0]:
a = agent.model.predict_on_batch(states)
b = agent.model(states)

np.mean(a == b)